## Medium SageMaker Analyzer

In this example we see how good my Medium articles on SageMaker are. We build a RAG based application by feeding some of my popular Medium articles (stored in sagemaker-articles) to a QA Chain. We've pre saved these articles in the sagemaker-articles directory as pdfs.

Requirements:

- OpenAI API Key: This is the LLM we use in this case

Documentation:

- https://python.langchain.com/docs/use_cases/question_answering/how_to/vector_db_qa

Additional Resources/Guides

- https://deci.ai/blog/retrieval-augmented-generation-using-langchain/

### Setup

In [ ]:
#!pip install --upgrade langchain

In [ ]:
#!pip install pypdf

In [ ]:
import os
os.environ['OPENAI_API_KEY'] = 'Enter your API Key here'
print(os.environ.get('OPENAI_API_KEY'))

In [ ]:
import langchain
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.embeddings.cache import CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI

### RAG Setup

We need the following to implement RAG with LangChain:

- <b>Storage</b>: A local store for our data we are providing for our RAG application. To scale up you can utilize other stores such as S3 as this gets larger
- <b>Embeddings model</b>: To create embedding out of the provided data, we use OpenAI Embeddings
- <b>Vector Store</b>: Store model embeddings, FAISS in this case
- <b>Chain</b>: Stitches together these different components, our LLM models is OpenAI in this case

In [ ]:
# where our embeddings will be stored
store = LocalFileStore("./cache/")

In [ ]:
# instantiate a loader: this loads our data, use PDF in this case
loader = PyPDFDirectoryLoader("sagemaker-articles/")

In [ ]:
# by default the PDF loader both loads and splits the documents for us
pages = loader.load_and_split()
print(len(pages))

In [ ]:
# instantiate embedding model
embeddings_model = OpenAIEmbeddings()

# pass in our vector store
embedder = CacheBackedEmbeddings.from_bytes_store(
    embeddings_model,
    store
)

In [ ]:
# create vector store, we use FAISS in this case
vector_store = FAISS.from_documents(pages, embedder)

### Chain Creation

In [ ]:
# this is the entire retrieval system
medium_qa_chain = RetrievalQA.from_chain_type(
    llm=ChatOpenAI(),
    retriever=vector_store.as_retriever(),
    return_source_documents=True,
    verbose=True
)

### Sample Inference

In [ ]:
sample_prompts = ["What does Ram Vegiraju write about?",
                 "What is Amazon SageMaker?",
                 "What is Amazon SageMaker Inference?",
                 "What are the different hosting options for Amazon SageMaker?",
                 "What is Serverless Inference with Amazon SageMaker?",
                 "What's the difference between Multi-Model Endpoints and Multi-Container Endpoints?",
                 "What SDKs can I use to work with Amazon SageMaker?"]

In [ ]:
for prompt in sample_prompts:
    response = medium_qa_chain({"query":prompt})
    print(response['result'])